In [6]:
import os
import math
import torch
import wandb
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.utilities.model_summary import ModelSummary
from torchvision import transforms

import matplotlib.pyplot as plt

from data.datamodule import Animal_DataModule
from models.model_cnn_mika import CatsDogsModel

### Loading Configuration

In the following steps, we will load the configuration settings using the `load_configuration` function. The configuration is stored in the `config` variable which will be used throughout the script.

In [2]:
from config.load_configuration import load_configuration
config = load_configuration()

PC Name: DESKTOP-MIKA
Loaded configuration from config/config_mika.yaml


### Setting Seeds for Reproducibility

To ensure comparable and reproducible results, we set the random seed using the `seed_everything` function from PyTorch Lightning. This helps in achieving consistent behavior across multiple runs of the notebook.

In [3]:
pl.seed_everything(config['seed'])
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"   # disable oneDNN optimizations for reproducibility

Seed set to 42


### Checking for GPU Devices

In this step, we check for the availability of GPU devices and print the device currently being used by PyTorch. This ensures that the computations are performed on the most efficient hardware available.

In [4]:
# Check if CUDA is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Torch Version: ', torch.__version__)
print('Using device: ', device)
if device.type == 'cuda':
    print('Cuda Version: ', torch.version.cuda)
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    torch.set_float32_matmul_precision('high')

Torch Version:  2.7.0+cu128
Using device:  cuda
Cuda Version:  12.8
NVIDIA GeForce GTX 1060 6GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


### Defining Transformations and Instantiating DataModule

In this step, we will define the necessary data transformations and initialize the `Animal_DataModule` with the provided configuration.

In [5]:
# TODO: Define transformations here

# Example for transformation
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((300, 300)),  # Resize images to match EfficientNet input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ImageNet normalization
])

dl = Animal_DataModule(config['path_to_data'])

### Creating the Model

In this step, we will define the model architecture and print its summary using the `ModelSummary` utility from PyTorch Lightning. This provides an overview of the model's layers, parameters, and structure.

In [7]:
model = CatsDogsModel()
print(ModelSummary(model, max_depth=-1))  

   | Name     | Type           | Params | Mode 
-----------------------------------------------------
0  | model    | Sequential     | 21.3 M | train
1  | model.0  | Conv2d         | 896    | train
2  | model.1  | ReLU           | 0      | train
3  | model.2  | MaxPool2d      | 0      | train
4  | model.3  | Conv2d         | 18.5 K | train
5  | model.4  | ReLU           | 0      | train
6  | model.5  | MaxPool2d      | 0      | train
7  | model.6  | Conv2d         | 73.9 K | train
8  | model.7  | ReLU           | 0      | train
9  | model.8  | MaxPool2d      | 0      | train
10 | model.9  | Flatten        | 0      | train
11 | model.10 | Linear         | 21.2 M | train
12 | model.11 | ReLU           | 0      | train
13 | model.12 | Dropout        | 0      | train
14 | model.13 | Linear         | 513    | train
15 | model.14 | Sigmoid        | 0      | train
16 | accuracy | BinaryAccuracy | 0      | train
-----------------------------------------------------
21.3 M    Trainable params
0

In [ ]:
# Initialize the Wandb logger
wandb_config = {
    'project_name': config['wandb_project_name'],
    'experiment_name': config['wandb_experiment_name'],
    'batch_size': config['batch_size'],
    'max_epochs': config['max_epochs'],
    'learning_rate': config['learning_rate'],
}

wandb_logger = WandbLogger(
    project=config['wandb_project_name'],
    name=config['wandb_experiment_name'],
    config=config
    # save_dir=os.path.join(config['path_to_data'], 'logs')
)

In [ ]:
# Initialize Trainer with wandb logger, using early stopping callback (https://lightning.ai/docs/pytorch/stable/common/early_stopping.html)
trainer = Trainer(
    max_epochs=config['max_epochs'], 
    default_root_dir='model/checkpoint/', #data_directory, 
    accelerator="auto", 
    devices="auto", 
    strategy="auto",
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, mode='min')], 
    logger=wandb_logger)

# Training of the model
trainer.fit(model=model, datamodule=dm)

# Finish wandb
wandb.finish()